In [135]:
#.venv/scripts/activate  ; no source. 
import xgboost as xgb
from matplotlib.pyplot import hist
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import pandas as pd

import numpy as np
import pandas as pd

RANDOM_SEED = 23


In [136]:
# Importing maindata
file_path = "C://Users/miste/Documents/Causal_ML/"
x = pd.read_stata(file_path + "maindata.dta", convert_categoricals=False)

# Importing laws_csv, cleaning it and merging with maindata
laws_csv = pd.read_csv(file_path + "When_Were_Laws.csv")
laws_csv = laws_csv[np.logical_not(np.isnan(laws_csv["FIPS"]))]

laws_csv = laws_csv.drop("State_Name", axis=1)
laws_csv = laws_csv.rename({'FIPS': 'stfips'}, axis=1) 

merged = pd.merge(laws_csv, x, on='stfips', how='outer')


In [137]:
# Super basic model, we only look at 1997
# We drop all of the NaN rows for the variables we are interested in. This leaves us with: 864922 observations

basic_merged = merged.copy()

basic_merged = basic_merged[basic_merged["Year_Implemented"].str.contains("always")==False]
basic_merged = basic_merged.replace("never", "1000000")  # Means that the indicator for year will always equal 1
basic_merged["Year_Implemented"] = basic_merged["Year_Implemented"].astype(int) # converting to int

basic_merged["year_indic"] = (basic_merged["year"] >= basic_merged["Year_Implemented"])  # indicator for if treatment has occured in state i
basic_merged["treatment"] = basic_merged["under19"] * basic_merged["year_indic"]

list_of_confounders = [ "fownu18", "fpovcut", "povll", "faminctm1", "a_maritl"] 
list_of_confounders += ["a_hga",  "anykids", "year", "stfips", "disability", "elig"] 

#list_of_confounders += ["noemp_insured", "disability"] 
# Dropping years  outside of [1995,2000] 
basic_merged = basic_merged[basic_merged["year"] <= 2000]
basic_merged = basic_merged[basic_merged["year"] >= 1995]

basic_merged = basic_merged[list_of_confounders + ["treatment", "pubonly", "insured", "privonly", "Year_Implemented"]]
basic_merged = basic_merged.dropna(axis = 0)

confounders_and_treat = basic_merged[list_of_confounders + ["treatment"]]
confounders_no_treat = basic_merged[list_of_confounders]

y_var = basic_merged["privonly"]

In [138]:
# Fitting Q

x_train, x_test, y_train, y_test = train_test_split(confounders_and_treat, y_var, test_size=0.2)

Q = HistGradientBoostingClassifier()
Q.fit(x_train, y_train)
score2 = Q.score(x_test,y_test)
print("Model Score: " + str(score2))   
                          
# Can fit g as well, normally get ~95-99%

Model Score: 0.805837797893959


In [139]:
#ATT(1997, 1997, 0) = 
# G_g = 1 if Year_Imp == 1997
# E(G_g) = % of those first treated in 1997
# Y_t = projected insured rate at time t
# Y_{g-1} (we assume delta = 0 here) = projected insured rate at time g - 1 = 1996
# C = 1 if never treated
# m_nev = E(Y_t^C - Y_{g-1}^C | X, C = 1) so we want to estimate Y_t and Y_1996 for those never treated. 

#Then:
# ATT(1997, 1997,0) = E[G_g / E(G_g) * (Y_t - Y_{g-1} - m_nev)]

t = 1997
base_year = 1996


basic_merged["t_indic"] = (basic_merged["Year_Implemented"] == 1997)
basic_merged["never_indic"] = (basic_merged["Year_Implemented"] == 1000000)


Yt = confounders_and_treat.copy()
Yg = confounders_and_treat.copy()

Yt["year"] = t
Yg["year"] = base_year



Yt = Q.predict_proba(Yt)[:,1]
Yg = Q.predict_proba(Yg)[:,1]

Yt_C = confounders_and_treat.copy()
Yg_C = confounders_and_treat.copy()

Yt_C["year"] = t
Yg_C["year"] = base_year

Yt_C["treatment"] = 0
Yg_C["treatment"] = 0

Yt_C = Q.predict_proba(Yt_C)[:,1]
Yg_C = Q.predict_proba(Yg_C)[:,1]

term_1 = basic_merged["t_indic"] / np.mean(basic_merged["t_indic"])
term_2 = (Yt - Yg - (Yt_C - Yg_C))


In [140]:
# Actually computing it now
term_1 = basic_merged["t_indic"] / np.mean(basic_merged["t_indic"])
term_2 = (Yt - Yg - (Yt_C - Yg_C))

mean = np.mean(term_1 * term_2)
ste = np.std(term_1 * term_2) / np.sqrt(Yt.shape[0])

print("Mean: " + str(mean))
print("STE: " + str(ste))

print("t: " + str( mean / ste))

#so we get + .02% lmao, at least its significant. 

Mean: 4.8824392437633997e-05
STE: 2.170430722799204e-05
t: 2.2495254939381426
